In [56]:
!pip install selenium
from selenium import webdriver
import time
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import os
cwd = os.getcwd()
print(cwd)

In [57]:
import os
import time


# Define folder name with timestamp
timestamp = time.strftime('%Y-%m-%d_%H-%M-%S')
folder_name = f'CDC_W_API_download_{timestamp}'

# Create folder
download_folder = os.path.join(r'C:\Users\khazi\Desktop\CDC WONDER API', folder_name)
os.makedirs(download_folder, exist_ok=True)


In [58]:
download_folder

'C:\\Users\\khazi\\Desktop\\CDC WONDER API\\CDC_W_API_download_2024-03-08_01-49-06'

In [59]:
https_links = [link.strip() for link in links if link.strip().startswith('https')]
print(https_links)

['https://wonder.cdc.gov/controller/saved/D158/D382F099', 'https://wonder.cdc.gov/controller/saved/D158/D382F100', 'https://wonder.cdc.gov/controller/saved/D158/D382F101', 'https://wonder.cdc.gov/controller/saved/D158/D382F102', 'https://wonder.cdc.gov/controller/saved/D158/D382F103', 'https://wonder.cdc.gov/controller/saved/D158/D382F105', 'https://wonder.cdc.gov/controller/saved/D158/D382F106', 'https://wonder.cdc.gov/controller/saved/D158/D382F107', 'https://wonder.cdc.gov/controller/saved/D158/D382F108', 'https://wonder.cdc.gov/controller/saved/D158/D382F109', 'https://wonder.cdc.gov/controller/saved/D158/D382F112', 'https://wonder.cdc.gov/controller/saved/D158/D382F111', 'https://wonder.cdc.gov/controller/saved/D158/D382F113', 'https://wonder.cdc.gov/controller/saved/D158/D382F114', 'https://wonder.cdc.gov/controller/saved/D158/D382F116', 'https://wonder.cdc.gov/controller/saved/D158/D382F117', 'https://wonder.cdc.gov/controller/saved/D158/D382F119', 'https://wonder.cdc.gov/contro

In [60]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Set Chrome options
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,
    "download.default_directory": download_folder,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    "profile.default_content_settings.popups": 0,
    "profile.content_settings.exceptions.automatic_downloads.*.setting": 1,
})

# Initialize WebDriver with Chrome options
driver = webdriver.Chrome(executable_path=r'C:\Users\khazi\Desktop\CDC WONDER API\chromedriver.exe', options=chrome_options)

# Read links from file
file_path = r'C:\Users\khazi\Desktop\CDC WONDER API\CDC_Wonder_API_links_new.txt'
with open(file_path, 'r') as file:
    links = file.readlines()

https_links = [link.strip() for link in links if link.strip().startswith('https')]

# Process each link
for link in https_links:
    driver.get(link)
    time.sleep(5)  # Time delay for the web page to load before accessing the frame

    # Find and click the "I Agree" button
    driver.find_element("xpath", ".//input[@name='action-I Agree']").click()

    time.sleep(5) # Time delay for the web page to load before accessing the frame

    # Find and click the "Export" button
    driver.find_element("xpath", ".//input[@value='Export']").click()

    time.sleep(25)

# Close the browser outside the loop
driver.quit()


C:\Users\khazi\AppData\Local\Temp\ipykernel_24132\2729204823.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Users\khazi\Desktop\CDC WONDER API\chromedriver.exe', options=chrome_options)


In [61]:
import csv
import os
import glob
import pandas as pd

# Input folder path containing text files
input_folder_path = download_folder

# Output Excel file path
output_excel_file_path = os.path.join(input_folder_path, 'output_merged.xlsx')

def process_text_file(input_file_path):
    # Read content from the input file
    with open(input_file_path, 'r') as input_file:
        content = input_file.read()

    # Find the index of the "---" separator
    separator_index = content.find("---")

    # Extract content above the separator
    new_content = content[:separator_index].strip()

    return new_content

def text_to_columns(input_csv, output_csv, delimiter='\t'):
    with open(input_csv, 'r') as infile, open(output_csv, 'w', newline='') as outfile:
        reader = csv.reader(infile, delimiter=delimiter)
        writer = csv.writer(outfile, delimiter=',')

        for row in reader:
            writer.writerow(row)

# Create an Excel writer object
with pd.ExcelWriter(output_excel_file_path, engine='xlsxwriter') as excel_writer:
    # Loop through all text files in the folder
    for i, file_path in enumerate(glob.glob(os.path.join(input_folder_path, '*.txt'))):
        file_name = os.path.basename(file_path)

        # Process each text file
        new_content = process_text_file(file_path)

        # Split the content into lines
        lines = new_content.strip().split('\n')

        # Create a DataFrame with the data
        df = pd.DataFrame([line.replace('"', '').split('\t') for line in lines])

        # Add a new row at the end with the original file name
        df.loc[len(df.index)] = [file_name] + [''] * (len(df.columns) - 1)

        # Write the DataFrame to a new sheet in the Excel file
        sheet_name = f"Sheet{i+1}"
        df.to_excel(excel_writer, sheet_name=sheet_name, index=False, header=False)

print("Content has been written to", output_excel_file_path)


Content has been written to C:\Users\khazi\Desktop\CDC WONDER API\CDC_W_API_download_2024-03-08_01-49-06\output_merged.xlsx


In [5]:
import pandas as pd

# Function to find the comment row, extract it, and set it as a new column "Category" in each sheet
def add_category_row(xls, sheet_name):
    # Load the sheet
    df = xls.parse(sheet_name)
    # Find the comment row ending with ".txt" and store it
    comment_row = df[df.apply(lambda x: x.astype(str).str.endswith('.txt').any(), axis=1)]
    if not comment_row.empty:
        # Extract the first matching value
        category_comment = comment_row.stack().loc[comment_row.stack().str.endswith('.txt')].values[0]
        # Add as a new column
        df['Category'] = category_comment
    else:
        df['Category'] = "No comment found"
    return df

# Load the Excel file (adjust the file path accordingly)
xls = pd.ExcelFile(r'C:\Users\khazi\Desktop\CDC WONDER API\CDC_W_API_download_2024-03-08_01-49-06\output_merged.xlsx')

# Get the names of all sheets in the Excel file
sheet_names = xls.sheet_names

# Apply the function to each sheet and update the dataframes
updated_dataframes = {sheet_name: add_category_row(xls, sheet_name) for sheet_name in sheet_names}

# Example: Show the first few rows of Sheet1 after updating
print(updated_dataframes['Sheet1'].head())


   Notes     Hispanic Origin Hispanic Origin Code Ten-Year Age Groups  \
0    NaN  Hispanic or Latino               2135-2         55-64 years   
1    NaN  Hispanic or Latino               2135-2         55-64 years   
2    NaN  Hispanic or Latino               2135-2         55-64 years   
3    NaN  Hispanic or Latino               2135-2         55-64 years   
4  Total  Hispanic or Latino               2135-2         55-64 years   

  Ten-Year Age Groups Code    Year  Year Code  Deaths  \
0                    55-64  2018.0     2018.0    28.0   
1                    55-64  2019.0     2019.0    37.0   
2                    55-64  2020.0     2020.0    80.0   
3                    55-64  2021.0     2021.0    73.0   
4                    55-64     NaN        NaN   218.0   

                                            Category  
0  10 yr age groups for MORTALITY for hispanic in...  
1  10 yr age groups for MORTALITY for hispanic in...  
2  10 yr age groups for MORTALITY for hispanic in... 

In [12]:
import pandas as pd

# Function to find the comment row, extract it, and set it as a new column "Category" in each sheet
def add_category_row(df):
    # Filter out rows where the value in the "Notes" column is "Total"
    df = df[df['Notes'] != 'Total']
    # Find the comment row ending with ".txt" and store it
    comment_row = df[df.apply(lambda x: x.astype(str).str.endswith('.txt').any(), axis=1)]
    if not comment_row.empty:
        # Extract the first matching value
        category_comment = comment_row.stack().loc[comment_row.stack().str.endswith('.txt')].values[0]
        # Add as a new column
        df['Category'] = category_comment
    else:
        df['Category'] = "No comment found"
    return df

# Load the Excel file (adjust the file path accordingly)
input_file = r'C:\Users\khazi\Desktop\CDC WONDER API\CDC_W_API_download_2024-03-08_01-49-06\output_merged.xlsx'
xls = pd.ExcelFile(input_file)

# Get the names of all sheets in the Excel file
sheet_names = xls.sheet_names

# Dictionary to store updated dataframes for all sheets
updated_dataframes = {}

# Loop through each sheet and apply the function
for sheet_name in sheet_names:
    # Load the sheet
    df = xls.parse(sheet_name)
    # Apply the function
    updated_df = add_category_row(df)
    # Store the updated dataframe
    updated_dataframes[sheet_name] = updated_df

# Save the updated dataframes to a new Excel file
output_file = 'updated_output.xlsx'
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, df in updated_dataframes.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Updated data saved to '{output_file}'.")


C:\Users\khazi\AppData\Local\Temp\ipykernel_40492\3840403781.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Category'] = category_comment
C:\Users\khazi\AppData\Local\Temp\ipykernel_40492\3840403781.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Category'] = category_comment
C:\Users\khazi\AppData\Local\Temp\ipykernel_40492\3840403781.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


Updated data saved to 'updated_output.xlsx'.


In [20]:
import pandas as pd

# Load the Excel file (adjust the file path accordingly)
input_file = r'updated_output.xlsx'
xls = pd.ExcelFile(input_file)

# Get the names of all sheets in the Excel file
sheet_names = xls.sheet_names

# Initialize a dictionary to store DataFrames for each sheet
dfs = {}

# Iterate through each sheet
for sheet_name in sheet_names:
    # Load the sheet into a DataFrame
    df = xls.parse(sheet_name)
    # Store the DataFrame in the dictionary
    dfs[sheet_name] = df

# Merge all DataFrames
merged_df = pd.concat(dfs.values(), ignore_index=True, sort=False)


In [21]:
merged_df

,Notes,Hispanic Origin,Hispanic Origin Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Year,Year Code,Deaths,Category,Population,...,Single Race 6,Single Race 6 Code,Age Adjusted Rate,Age Adjusted Rate Lower 95% Confidence Interval,Age Adjusted Rate Upper 95% Confidence Interval,Age Adjusted Rate Standard Error,State,State Code,Gender,Gender Code
0,NaN,Hispanic or Latino,2135-2,55-64 years,55-64,2018.0,2018.0,28.0,10 yr age groups for MORTALITY for hispanic in...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Hispanic or Latino,2135-2,55-64 years,55-64,2019.0,2019.0,37.0,10 yr age groups for MORTALITY for hispanic in...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Hispanic or Latino,2135-2,55-64 years,55-64,2020.0,2020.0,80.0,10 yr age groups for MORTALITY for hispanic in...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Hispanic or Latino,2135-2,55-64 years,55-64,2021.0,2021.0,73.0,10 yr age groups for MORTALITY for hispanic in...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Hispanic or Latino,2135-2,65-74 years,65-74,2018.0,2018.0,28.0,10 yr age groups for MORTALITY for hispanic in...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,NaN,NaN,NaN,NaN,NaN,2019.0,2019.0,77.0,AGE55UPSUIFEMALE and male all races in Central...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Indiana,18.0,Male,M
728,NaN,NaN,NaN,NaN,NaN,2020.0,2020.0,71.0,AGE55UPSUIFEMALE and male all races in Central...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Indiana,18.0,Male,M
729,NaN,NaN,NaN,NaN,NaN,2021.0,2021.0,70.0,AGE55UPSUIFEMALE and male all races in Central...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Indiana,18.0,Male,M
730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGE55UPSUIFEMALE and male all races in Central...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
"C:\Users\khazi\Desktop\merged.xlsx"

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3267990333.py, line 1)

In [23]:
# Save the merged DataFrame to a new Excel file
output_file = r'C:\Users\khazi\Desktop\merged.xlsx'
merged_df.to_excel(output_file, index=False)

print(f"Merged data saved to '{output_file}'.")

Merged data saved to 'C:\Users\khazi\Desktop\merged.xlsx'.
